In [2]:
import numpy as np

In [3]:
# Data from the problem set For AHS 1 to 12
improve_rate = np.array([0, 0.00032, 0.001134, 0.00311, 0.006721, 0.010747, 0.014862, 0.024625, 0.033571, 0.040201, 0.030801, 0.032320, 0.00032, 0.001134, 0.00311, 0.006721, 0.010747, 0.014862, 0.024625, 0.033571, 0.040201, 0.030801, 0.03232])
deteriorate_rate = np.array([0.001013, 0.000961, 0.001473, 0.003199, 0.005537, 0.008135, 0.010858, 0.02387, 0.0313, 0.037877, 0.031978, 0])
death_rate = np.array([0.000123, 0.000172, 0.000245, 0.000319, 0.000417, 0.000564, 0.00076, 0.001029, 0.001372, 0.001862, 0.00245, 0.003553])
transplant_prob = np.array([0.0001, 0.0002, 0.0002, 0.0003, 0.0004, 0.0006, 0.0008, 0.0010, 0.0014, 0.0019, 0.0024, 0.0035])
post_trans_days_all=np.array([3747,3623.25,3502.5,3380,3259.25,3137.5,3014,2898.25,2776.5,2661.25,2537,2417.25])

In [4]:
#rate to Probabilities
prob_improv_rate=1-np.exp(-improve_rate)
prob_det_rate=1-np.exp(-deteriorate_rate)
prob_death_rate=1-np.exp(-death_rate)

In [5]:
#Probability Matrix 
def probability_matrix(threshold_policy,prob_improv_rate,prob_det_rate,prob_death_rate,transplant_prob):
    P=np.zeros((14,14)) #probability matrix initialization

    threshold_policy=5 #threshold Policy
    P[13][13]=1
    P[12][12]=1
    for i in range(0,12): 
            P[i][i+1]=prob_det_rate[i]
            P[i][12]=prob_death_rate[i]
            P[i][i-1]=prob_improv_rate[i]
            if i+1>=threshold_policy:
                P[i][13]=transplant_prob[i]

            P[i][i]=1-(P[i][i+1]+P[i][12]+P[i][i-1]+P[i][13])
    return P

In [6]:
#likelihood and pre-transplant expected days calculation
def transient_analysis(P):
    Q=P[0:12,0:12]
    R=P[0:12,12:14]
    I=np.identity(12)

    inv=np.linalg.inv(I-Q)
    ones = np.ones((12,1))
    pre_trans_days=np.dot(inv,ones) #partb
    likelihood=np.dot(inv,R) #partc
    
    return pre_trans_days, likelihood


In [7]:
def total_expectation(threshold_policy,prob_improv_rate,prob_det_rate,prob_death_rate,transplant_prob,post_trans_days_all):

    P=probability_matrix(threshold_policy,prob_improv_rate,prob_det_rate,prob_death_rate,transplant_prob)
    pre_trans_days, likelihood=transient_analysis(P)
    
    #set up post_trans_days array
    post_trans_days_threshold_range=post_trans_days_all[threshold_policy-1:12]
    zeros=np.zeros(threshold_policy-1)
    post_trans_days_threshold=np.append(zeros,post_trans_days_threshold_range)
    post_trans_days_threshold=np.array([post_trans_days_threshold])

    #total expectation 
    Total_expec=np.multiply(np.array([likelihood[:,1]]).T,post_trans_days_threshold.T)+pre_trans_days #PART D
    return Total_expec

In [8]:
#given a threshold policy of 5 or up
x=total_expectation(1,prob_improv_rate,prob_det_rate,prob_death_rate,transplant_prob,post_trans_days_all)
x 

array([[4814.39747331],
       [4395.62154874],
       [3987.92127642],
       [3645.31466661],
       [3349.15377123],
       [3059.25431756],
       [2760.19616688],
       [2444.33041775],
       [2157.6753969 ],
       [1889.05954139],
       [1631.06439776],
       [1410.76624299]])

In [9]:
#given for all thresehold_policy
x=[]
y=[total_expectation(i,prob_improv_rate,prob_det_rate,prob_death_rate,transplant_prob,post_trans_days_all) for i in range(1,14)]
x=np.stack(y,axis=0)
x.reshape((13,12))


array([[4814.39747331, 4395.62154874, 3987.92127642, 3645.31466661,
        3349.15377123, 3059.25431756, 2760.19616688, 2444.33041775,
        2157.6753969 , 1889.05954139, 1631.06439776, 1410.76624299],
       [4384.01523781, 4395.62154874, 3987.92127642, 3645.31466661,
        3349.15377123, 3059.25431756, 2760.19616688, 2444.33041775,
        2157.6753969 , 1889.05954139, 1631.06439776, 1410.76624299],
       [4384.01523781, 3928.89902703, 3987.92127642, 3645.31466661,
        3349.15377123, 3059.25431756, 2760.19616688, 2444.33041775,
        2157.6753969 , 1889.05954139, 1631.06439776, 1410.76624299],
       [4384.01523781, 3928.89902703, 3439.69279181, 3645.31466661,
        3349.15377123, 3059.25431756, 2760.19616688, 2444.33041775,
        2157.6753969 , 1889.05954139, 1631.06439776, 1410.76624299],
       [4384.01523781, 3928.89902703, 3439.69279181, 2956.08904494,
        3349.15377123, 3059.25431756, 2760.19616688, 2444.33041775,
        2157.6753969 , 1889.05954139, 1631.0